### Libararies Importing

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

### BigQuery Connection

In [2]:
credentials = service_account.Credentials.from_service_account_file('E:/SoftwareBackup/restaurant-market-analysis-ed33d4754012.json')
project_id = 'restaurant-market-analysis'
client = bigquery.Client(credentials=credentials, project=project_id)

### Data Loading

In [3]:
dataset_id = 'Restaurant'
tables = {
    'restaurant_data': 'restaurant_data',
    'restaurant_checkin': 'restaurant_checkin',
    'restaurant_tips': 'restaurant_tips'
}

# Load Data
def load_data(table_name):
    query = f"""
    SELECT * FROM `{dataset_id}.{table_name}`;
    """
    return client.query(query).to_dataframe()


In [4]:
restaurant_data = load_data(tables['restaurant_data'])

In [5]:
restaurant_checkin = load_data(tables['restaurant_checkin'])

In [6]:
restaurant_tips = load_data(tables['restaurant_tips'])

In [7]:
restaurant_data

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,NoiseLevel,GoodForMeal,BusinessAcceptsBitcoin,Smoking,Music,GoodForDancing,BestNights,BYOB,Corkage,Parking
0,doN8QSdGtlIAW7BP8afZEg,McDonald's,109 E Hwy 50,O'Fallon,IL,62269,38.584549,-89.909163,1.5,45,...,None,"{'dessert': False, 'latenight': False, 'lunch'...",<NA>,None,None,<NA>,None,<NA>,<NA>,False
1,e8VlQHJYDyfMo5J--qyY9g,Krispy Kreme,3113 W Kennedy Blvd,Tampa,FL,33609,27.945087,-82.493933,3.0,87,...,'average',None,<NA>,None,None,<NA>,None,<NA>,<NA>,True
2,dRNrAjTbe9DCdpJUIW4c0g,Mr Lee's Asian Gourmet,1134 N Gravel Pike,Zieglerville,PA,19492,40.275734,-75.478052,3.5,27,...,'average',None,<NA>,None,None,<NA>,None,<NA>,<NA>,<NA>
3,2l1rAQ3tonK68LQEKtEiGA,The Market at Del Val,2100 Lower State Rd,Doylestown,PA,18901,40.293535,-75.150953,4.0,16,...,None,None,False,None,None,<NA>,None,<NA>,<NA>,True
4,cHrzPYXVLd1BEXWqi6Ug6Q,Ark Brewery,106 Ark Rd,Lumberton,NJ,08048,39.941709,-74.852849,3.0,35,...,None,None,<NA>,None,None,<NA>,None,<NA>,<NA>,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52281,7Qcdxav_IcbDvn_oYAVsKw,Italian Gardens Restaurant,8336 E Washington St,Indianapolis,IN,46219,39.773836,-86.019973,3.0,35,...,'quiet',None,<NA>,None,None,<NA>,None,<NA>,<NA>,True
52282,AsMmX7njOzFkWsJQ1jMVCg,The Boozy Pig,3255 West Cypress St,Tampa,FL,33607,27.952304,-82.497093,4.5,134,...,None,"{'dessert': False, 'latenight': False, 'lunch'...",<NA>,None,None,<NA>,None,<NA>,<NA>,True
52283,prjS_8VAw65e32UtoCj5AA,Seasons Pizza,1524 Philadelphia Pike,Wilmington,DE,19809,39.784551,-75.477870,2.0,28,...,'average',"{'dessert': False, 'latenight': False, 'lunch'...",<NA>,None,None,<NA>,None,<NA>,<NA>,True
52284,Kyey3rqsPuNOgGlXuiqezQ,Chipotle Mexican Grill,3696 W Gandy Blvd,Tampa,FL,33611,27.893439,-82.505866,2.0,100,...,'average',"{'dessert': False, 'latenight': False, 'lunch'...",<NA>,None,None,<NA>,None,<NA>,<NA>,True


### Data Preprocessing

#### 1. Data Understanding

In [8]:
# check the data types and the size of the dataframe
print(restaurant_data.dtypes)
print(restaurant_data.shape)
print(restaurant_checkin.dtypes)
print(restaurant_checkin.shape)
print(restaurant_tips.dtypes)
print(restaurant_tips.shape)

business_id        object
name               object
address            object
city               object
state              object
                   ...   
GoodForDancing    boolean
BestNights         object
BYOB              boolean
Corkage           boolean
Parking           boolean
Length: 777, dtype: object
(52286, 777)
business_id                             object
date                       datetime64[us, UTC]
total_number_of_checkin                  Int64
restaurant                             boolean
year                                     Int64
month                                    Int64
day                                      Int64
hour                                     Int64
dtype: object
(8515582, 8)
user_id                          object
business_id                      object
text                             object
date                datetime64[us, UTC]
compliment_count                  Int64
dtype: object
(649181, 5)


#### 2. Data Cleaning

In [9]:
# Remove duplicates
restaurant_data.drop_duplicates(subset=['business_id'], inplace=True)
restaurant_checkin.drop_duplicates(inplace=True)
restaurant_tips.drop_duplicates(inplace=True)

In [10]:
# Process missing value
print("Restaurant Data Missing Values:\n", restaurant_data.isnull().sum())
print("Restaurant Checkin Missing Values:\n", restaurant_checkin.isnull().sum())
print("Restaurant Tips Missing Values:\n", restaurant_tips.isnull().sum())

# The field to be analyzed has no missing value 
# restaurant_data.fillna({
#     'stars': restaurant_data['stars'].mean(),
#     'hours': 'Unknown'
# }, inplace=True)

Restaurant Data Missing Values:
 business_id           0
name                  0
address             443
city                  0
state                 0
                  ...  
GoodForDancing    48691
BestNights        47784
BYOB              48075
Corkage           48865
Parking            6659
Length: 777, dtype: int64
Restaurant Checkin Missing Values:
 business_id                0
date                       0
total_number_of_checkin    0
restaurant                 0
year                       0
month                      0
day                        0
hour                       0
dtype: int64
Restaurant Tips Missing Values:
 user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64


In [11]:
missing_stars = restaurant_data['stars'].isna()
missing_reviews = restaurant_data['hours'].isna()

missing_data = restaurant_data[missing_stars | missing_reviews]
print(missing_data)

                  business_id                         name  \
2      dRNrAjTbe9DCdpJUIW4c0g       Mr Lee's Asian Gourmet   
5      15IzVzKuvUCrpM1AFvDT_A                    Brew Haha   
10     Tr15iwMqN7b5AmlPJ71VgA                Crawling Crab   
11     lVlOhVi4KPR6atfQL7I_TA           Taste of St. Louis   
16     z_Esu8xXjlLL55kv3rzIww          Mama Mea's Pizzaria   
...                       ...                          ...   
52201  V5vBQ_LyE99i4EcfjAtyVw              El Amigo Market   
52210  nseRx1grlaDvx_aDliInXw    Canton Chinese Restaurant   
52215  IjulL_5PvUyJJyIHNQtu-w          Kala' Bites & Brews   
52257  2vbbxo8E4z_41e9E3WgNRw                        Play2   
52267  AyJV8HdltxPgcHclmCgQeg  St Cecilia Mexican Fish Fry   

                        address                   city state postal_code  \
2            1134 N Gravel Pike           Zieglerville    PA       19492   
5                 27 Trolley Sq             Wilmington    DE       19806   
10                         

In [12]:
# Check for matches of state, city, and latitude and longitude
import googlemaps


#### 3. Data Transformation & Feature Construction

In [13]:
# Country
canadian_provinces = ['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT']
us_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
             'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
             'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
             'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
             'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

def assign_country(state):
    if state in canadian_provinces:
        return 'Canada'
    elif state in us_states:
        return 'USA'
    else:
        return 'Unknown'

restaurant_data['country'] = restaurant_data['state'].apply(assign_country)

restaurant_data[['state', 'country']]


,state,country
0,IL,USA
1,FL,USA
2,PA,USA
3,PA,USA
4,NJ,USA
...,...,...
52281,IN,USA
52282,FL,USA
52283,DE,USA
52284,FL,USA


In [14]:
# Month to Week
restaurant_checkin['date'] = pd.to_datetime(restaurant_checkin['date'])
restaurant_checkin['day_of_week'] = restaurant_checkin['date'].dt.day_name()
print(restaurant_checkin[['date', 'day_of_week']].head())

                       date day_of_week
0 2022-01-09 00:59:00+00:00      Sunday
1 2022-01-09 01:41:51+00:00      Sunday
2 2022-01-15 00:34:45+00:00    Saturday
3 2022-01-06 15:20:21+00:00    Thursday
4 2022-01-09 18:46:52+00:00      Sunday


In [15]:
# Hour Unit
restaurant_checkin['hour'] = restaurant_checkin['date'].dt.hour
restaurant_checkin['minute'] = restaurant_checkin['date'].dt.minute
restaurant_checkin['hour_continuous'] = restaurant_checkin['hour'] + restaurant_checkin['minute'] / 60.0
print(restaurant_checkin[['date', 'hour', 'minute', 'hour_continuous']].head())

                       date  hour  minute  hour_continuous
0 2022-01-09 00:59:00+00:00     0      59         0.983333
1 2022-01-09 01:41:51+00:00     1      41         1.683333
2 2022-01-15 00:34:45+00:00     0      34         0.566667
3 2022-01-06 15:20:21+00:00    15      20        15.333333
4 2022-01-09 18:46:52+00:00    18      46        18.766667


In [16]:
# Checkin Counts Every Two Hours
restaurant_checkin['time_slot'] = (restaurant_checkin['hour'] // 2) * 2
checkin_counts = restaurant_checkin.groupby(['business_id', 'time_slot']).size().reset_index(name='checkin_count')
print(checkin_counts.head(10))

              business_id  time_slot  checkin_count
0  ---kPU91CF4Lq2-WlRu9Lw         14              1
1  ---kPU91CF4Lq2-WlRu9Lw         16              2
2  ---kPU91CF4Lq2-WlRu9Lw         20              6
3  ---kPU91CF4Lq2-WlRu9Lw         22              2
4  --0iUa4sNDFiZFrAdIWhZQ          0              2
5  --0iUa4sNDFiZFrAdIWhZQ          6              1
6  --0iUa4sNDFiZFrAdIWhZQ         14              1
7  --0iUa4sNDFiZFrAdIWhZQ         16              1
8  --0iUa4sNDFiZFrAdIWhZQ         20              2
9  --0iUa4sNDFiZFrAdIWhZQ         22              3


#### 4. Data Merging | Feature Selection

In [28]:
# Data sets are too redundant to be merged
# restaurant_integrity = pd.merge(restaurant_data, restaurant_checkin, on='business_id', how='left')
# restaurant_integrity = pd.merge(restaurant_integrity, restaurant_tips, on='business_id', how='left')
# restaurant_integrity

In [30]:
# Data Splitting
retain_columns = ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'country']
simplified_restaurant_data = restaurant_data[retain_columns]

remaining_columns = [col for col in restaurant_data.columns if col not in retain_columns and col != 'business_id']
remaining_columns.insert(0, 'business_id')
restaurant_attributes = restaurant_data[remaining_columns]

print(simplified_restaurant_data.head())
print(restaurant_attributes.head())

              business_id                    name              address  \
0  doN8QSdGtlIAW7BP8afZEg              McDonald's         109 E Hwy 50   
1  e8VlQHJYDyfMo5J--qyY9g            Krispy Kreme  3113 W Kennedy Blvd   
2  dRNrAjTbe9DCdpJUIW4c0g  Mr Lee's Asian Gourmet   1134 N Gravel Pike   
3  2l1rAQ3tonK68LQEKtEiGA   The Market at Del Val  2100 Lower State Rd   
4  cHrzPYXVLd1BEXWqi6Ug6Q             Ark Brewery           106 Ark Rd   

           city state postal_code   latitude  longitude  stars  review_count  \
0      O'Fallon    IL       62269  38.584549 -89.909163    1.5            45   
1         Tampa    FL       33609  27.945087 -82.493933    3.0            87   
2  Zieglerville    PA       19492  40.275734 -75.478052    3.5            27   
3    Doylestown    PA       18901  40.293535 -75.150953    4.0            16   
4     Lumberton    NJ       08048  39.941709 -74.852849    3.0            35   

   is_open country  
0        1     USA  
1        1     USA  
2        1 

#### 5. Data Sorting

In [25]:
sorted_restaurant_data = restaurant_data.sort_values(by='stars', ascending=False)
sorted_restaurant_data

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,BusinessAcceptsBitcoin,Smoking,Music,GoodForDancing,BestNights,BYOB,Corkage,Parking,country,price_range
10553,rmeq7nfiry0HFQD-on9tNw,Mamita's Hot Tamales,None,New Orleans,LA,70112,29.951066,-90.071532,5.0,12,...,<NA>,None,None,<NA>,None,<NA>,<NA>,True,USA,1
17372,brmx8fQ7JQMHgfU_Req7HA,Gonzo's Smokehouse & BBQ,"12325 River Rd, Ste B",Luling,LA,70070,29.930066,-90.342651,5.0,19,...,False,None,None,<NA>,None,<NA>,<NA>,False,USA,None
7861,qHU175eb4hSSs-_981ja0A,Buckets Eatery,15024 Madeira Way,Madeira Beach,FL,33708,27.801848,-82.801260,5.0,43,...,<NA>,None,None,<NA>,None,<NA>,<NA>,<NA>,USA,None
11574,OSyRjc5SLixdhnvpsqg5MA,3 Southern Girls,4402 Jefferson Hwy,New Orleans,LA,70121,29.959284,-90.171977,5.0,21,...,<NA>,None,None,<NA>,None,<NA>,<NA>,<NA>,USA,None
31212,OQSihGXRZqKMeW5vSNVwhQ,Kobachi,125-200 Festival Lane,Sherwood Park,AB,T8A 4Y8,53.526576,-113.299004,5.0,9,...,<NA>,None,None,<NA>,None,True,True,False,Canada,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27580,otEpgG6DdLiKtKtQyfM70g,Popeye's Fried Chicken,4100 George J Bean Pkwy,Tampa,FL,33607,27.982758,-82.531764,1.0,5,...,<NA>,None,None,<NA>,None,<NA>,<NA>,False,USA,1
8525,8NCYYDGGPtmL4NxQxKUKBA,Burger King,5921 Rising Sun Ave,Philadelphia,PA,19120,40.044244,-75.101804,1.0,5,...,<NA>,None,None,<NA>,None,<NA>,<NA>,True,USA,1
16139,heYRwsl6JfpgePIkNVYXBQ,China Dragon,5677 Park St N,St. Petersburg,FL,33709,27.823363,-82.752567,1.0,9,...,<NA>,None,None,<NA>,None,<NA>,<NA>,<NA>,USA,None
1962,NQASJJzArkdQaXCl6JKOpA,Chipotle Mexican Grill,789 Delsea Dr N,Glassboro,NJ,08028,39.721945,-75.116001,1.0,20,...,<NA>,None,None,<NA>,None,<NA>,<NA>,<NA>,USA,None


### Data Storage

In [26]:
# Local Storage
sorted_restaurant_data.to_csv('E:/SoftwareBackup/restaurant_data_processed.csv', index=False)
restaurant_checkin.to_csv('E:/SoftwareBackup/restaurant_checkin_processed.csv', index=False)
restaurant_tips.to_csv('E:/SoftwareBackup/restaurant_tips_processed.csv', index=False)

In [27]:
# BigQuery Storage
def save_to_bigquery(df, table_name):
   destination = f"{dataset_id}.{table_name}_processed"
   job = client.load_table_from_dataframe(df, destination)
   job.result()

save_to_bigquery(sorted_restaurant_data, 'restaurant_data')
save_to_bigquery(restaurant_tips, 'restaurant_tips')
save_to_bigquery(restaurant_checkin, 'restaurant_checkin')

In [29]:
save_to_bigquery(checkin_counts, 'checkin_counts')

In [31]:
save_to_bigquery(simplified_restaurant_data, 'simplified_restaurant_data')

In [32]:
save_to_bigquery(restaurant_attributes, 'restaurant_attributes')